<a href="https://colab.research.google.com/github/Guylord/APHRC_DSE_Inspire_Hackathon_2024/blob/main/APHRC_DSE_Inspire_Hackathon_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
pip install recordlinkage

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import pandas as pd
import numpy as np
import recordlinkage as rl

In [13]:
hdss_df = pd.read_csv(r'C:\Users\hp\Desktop\Records\APHRC_DSE_Inspire_Hackathon_2024\synthetic_facility_v3.csv')
facility_df = pd.read_csv(r'C:\Users\hp\Desktop\Records\APHRC_DSE_Inspire_Hackathon_2024\synthetic_facility_v3.csv')

print('HDSS INFO')
hdss_df.info()
print('\n\nFACILITY INFO')
facility_df.info()

HDSS INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2902 entries, 0 to 2901
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   recnr       2902 non-null   int64 
 1   firstname   2902 non-null   object
 2   lastname    2835 non-null   object
 3   petname     2511 non-null   object
 4   dob         2902 non-null   object
 5   sex         2902 non-null   int64 
 6   nationalid  2902 non-null   object
 7   patientid   2902 non-null   int64 
 8   visitdate   2902 non-null   object
dtypes: int64(3), object(6)
memory usage: 204.2+ KB


FACILITY INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2902 entries, 0 to 2901
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   recnr       2902 non-null   int64 
 1   firstname   2902 non-null   object
 2   lastname    2835 non-null   object
 3   petname     2511 non-null   object
 4   dob         2902 non-null

# *FACILITY dataset*

In [14]:
facility_df.head()

,recnr,firstname,lastname,petname,dob,sex,nationalid,patientid,visitdate
0,2,Fatuma,NaN,Zaina,24-08-2017 00:00,2,N_ID_5000,2069,10-09-2018
1,3,Gloria,Rashida,NaN,11-07-1993 00:00,2,N_ID_11861,2079,14-12-2022
2,4,Ali,Hakram,Igomu,17-05-2014 00:00,1,N_ID_11864,2080,09-06-2023
3,5,Nakalema,NaN,Nkwanga,27-02-2026 00:00,2,N_ID_11867,2081,07-02-2019
4,6,Asuman,Sempa,Aguti,02-03-2002 00:00,1,N_ID_11870,2082,18-08-2020


In [15]:
facility_df.set_index('recnr', inplace=True)

In [16]:
from recordlinkage.preprocessing import clean

for col in facility_df.select_dtypes('object').columns:
  clean(facility_df[col], replace_by_none='[^ \\-\\_A-Za-z0-9]+', remove_brackets=True)

In [17]:
facility_df['fullname'] = facility_df['firstname'].fillna('') + ' ' + facility_df['lastname'].fillna('') + ' ' + facility_df['petname'].fillna('')

In [20]:
facility_df['dob'] = pd.to_datetime(facility_df['dob'])
facility_df['visitdate'] = pd.to_datetime(facility_df['visitdate'], format='%d-%m-%Y')

In [21]:
facility_df.isna().sum()

firstname       0
lastname       67
petname       391
dob             0
sex             0
nationalid      0
patientid       0
visitdate       0
fullname        0
dtype: int64

In [22]:
facility_df.duplicated().sum()

0

In [23]:
facility_df[facility_df.duplicated(subset=['fullname', 'dob', 'sex'], keep=False)].sort_values('fullname').head(10)

,firstname,lastname,petname,dob,sex,nationalid,patientid,visitdate,fullname
recnr,,,,,,,,,
2686,Abubakali,Lunkumu,Jjunju,1934-08-07,1,N_ID_13055,4754,2019-06-01,Abubakali Lunkumu Jjunju
2583,Abubakali,Lunkumu,Jjunju,1934-08-07,1,N_ID_12749,4651,2022-12-21,Abubakali Lunkumu Jjunju
1162,Abuneli,Lawrence,Asega,1959-08-11,1,N_ID_8444,3231,2019-09-20,Abuneli Lawrence Asega
575,Abuneli,Lawrence,Asega,1959-08-11,1,N_ID_6689,2645,2022-07-03,Abuneli Lawrence Asega
2533,Aidah,Fatuma,Aya,2011-12-13,2,N_ID_12599,4601,2020-06-23,Aidah Fatuma Aya
2590,Aidah,Fatuma,Aya,2011-12-13,2,N_ID_12770,4658,2020-04-19,Aidah Fatuma Aya
629,Akello,Jesca,Barungi,1937-12-20,2,N_ID_6851,2699,2023-06-08,Akello Jesca Barungi
1299,Akello,Jesca,Barungi,1937-12-20,2,N_ID_8855,3368,2022-08-13,Akello Jesca Barungi
1633,Alex,Idinda,Nakato,1967-10-08,1,N_ID_9860,3702,2020-01-10,Alex Idinda Nakato


In [24]:
facility_df.duplicated(subset='nationalid', keep=False).sum()

0



* It's evident that some records have the same entries for 'firstname,' 'lastname,' 'petname,' and 'dob,' implying that they refer to the same entity but with different 'nationalid' entries during various visits to the facility. This suggests that individuals visiting the facility did not retain a single 'nationalid,' rendering 'nationalid' unusable as a unique identifier

* Individuals have different 'patientid' entries at each visit to the health facility






# *HDSS dataset*

In [25]:
hdss_df.head()

,recnr,firstname,lastname,petname,dob,sex,nationalid,patientid,visitdate
0,2,Fatuma,NaN,Zaina,24-08-2017 00:00,2,N_ID_5000,2069,10-09-2018
1,3,Gloria,Rashida,NaN,11-07-1993 00:00,2,N_ID_11861,2079,14-12-2022
2,4,Ali,Hakram,Igomu,17-05-2014 00:00,1,N_ID_11864,2080,09-06-2023
3,5,Nakalema,NaN,Nkwanga,27-02-2026 00:00,2,N_ID_11867,2081,07-02-2019
4,6,Asuman,Sempa,Aguti,02-03-2002 00:00,1,N_ID_11870,2082,18-08-2020


In [26]:
hdss_df.set_index('recnr', inplace=True)

In [27]:
for col in hdss_df.select_dtypes('object').columns:
  clean(hdss_df[col], replace_by_none='[^ \\-\\_A-Za-z0-9]+', remove_brackets=True)

In [28]:
hdss_df['fullname'] = hdss_df['firstname'].fillna('') + ' ' + hdss_df['lastname'].fillna('') + ' ' + hdss_df['petname'].fillna('')

In [29]:
hdss_df['dob'] = pd.to_datetime(hdss_df['dob'])

C:\Users\hp\AppData\Local\Temp\ipykernel_24636\3528711419.py:1: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  hdss_df['dob'] = pd.to_datetime(hdss_df['dob'])


In [30]:
hdss_df.isna().sum()

firstname       0
lastname       67
petname       391
dob             0
sex             0
nationalid      0
patientid       0
visitdate       0
fullname        0
dtype: int64

In [31]:
hdss_df.duplicated().sum()

0

In [32]:
mask = hdss_df.duplicated(subset=['fullname', 'dob', 'sex'], keep=False)
hdss_df[mask].sort_values('fullname')

,firstname,lastname,petname,dob,sex,nationalid,patientid,visitdate,fullname
recnr,,,,,,,,,
2686,Abubakali,Lunkumu,Jjunju,1934-08-07,1,N_ID_13055,4754,01-06-2019,Abubakali Lunkumu Jjunju
2583,Abubakali,Lunkumu,Jjunju,1934-08-07,1,N_ID_12749,4651,21-12-2022,Abubakali Lunkumu Jjunju
1162,Abuneli,Lawrence,Asega,1959-08-11,1,N_ID_8444,3231,20-09-2019,Abuneli Lawrence Asega
575,Abuneli,Lawrence,Asega,1959-08-11,1,N_ID_6689,2645,03-07-2022,Abuneli Lawrence Asega
2533,Aidah,Fatuma,Aya,2011-12-13,2,N_ID_12599,4601,23-06-2020,Aidah Fatuma Aya
...,...,...,...,...,...,...,...,...,...
1028,Zawumina,Nakaziba,Kamau,1959-09-21,2,N_ID_8045,3097,23-09-2020,Zawumina Nakaziba Kamau
2901,Ziriya,Kauma,Kato,1963-11-11,2,N_ID_13682,4963,27-12-2018,Ziriya Kauma Kato
1310,Ziriya,Kauma,Kato,1963-11-11,2,N_ID_8888,3379,20-12-2021,Ziriya Kauma Kato


In [34]:
hdss_df.duplicated(subset='hdsshhid').sum()

KeyError: Index(['hdsshhid'], dtype='object')

In [ ]:
#indexer = rl.Index()
#indexer.block(left_on='dob', right_on='dob')
#pairs = indexer.index(facility_df, hdss_df)

In [ ]:
#indexer = rl.Index()
#indexer.block(left_on='fullname', right_on='fullname')
#pairs = indexer.index(facility_df, hdss_df)

In [35]:
indexer = rl.Index()
indexer.block(left_on='firstname', right_on='firstname')
pairs = indexer.index(hdss_df, facility_df)

In [ ]:
#indexer = rl.Index()
#indexer.block(left_on='sex', right_on='sex')
#pairs = indexer.index(facility_df, hdss_df)

In [36]:
compare = rl.Compare()

compare.string('firstname', 'firstname', method='jarowinkler', label='firstname')
compare.string('fullname', 'fullname', method='lcs', threshold = 0.70, label='fullname')
compare.date('dob', 'dob', label='dob')
compare.exact('sex', 'sex', label='sex')


potential_matches=compare.compute(pairs, hdss_df, facility_df)
potential_matches

firstname  fullname  dob  sex
recnr_1 recnr_2                               
2       2              1.0       1.0    1    1
        261            1.0       0.0    0    1
        277            1.0       0.0    0    1
        492            1.0       0.0    0    1
        570            1.0       0.0    0    1
...                    ...       ...  ...  ...
2866    2866           1.0       1.0    1    1
2873    2873           1.0       1.0    1    1
2875    2875           1.0       1.0    1    1
2880    2880           1.0       1.0    1    1
2888    2888           1.0       1.0    1    1

[26264 rows x 4 columns]

In [37]:
potential_matches.reset_index().duplicated(subset=['recnr_1']).sum()

23362

In [38]:
potential_matches.reset_index().duplicated(subset=['recnr_2']).sum()

23362

In [39]:
#lr=recordlinkage.LogisticRegressionClassifier()
#nb=recordlinkage.NaiveBayesClassifier()
#ecm=recordlinkage.ECMClassifier()
kmeans=rl.KMeansClassifier()
kmeans.fit(potential_matches)
matched_indices = kmeans.predict(potential_matches)
len(matched_indices)

4708

In [ ]:
hdss_facility = pd.concat([hdss_df.loc[matched_indices.get_level_values(0)], facility_df.loc[matched_indices.get_level_values(1)]])
hdss_facility.sort_values('firstname').head(30)

,firstname,lastname,petname,dob,sex,nationalid,hdssid,hdsshhid,fullname,patientid,visitdate
3482,Ababu,Crispus,Lwasa,2006-04-23,1,NaN,I23482,HH103482,Ababu Crispus Lwasa,NaN,NaT
1209,Ababu,Crispus,Lwasa,2006-04-23,1,N_ID_8585,NaN,NaN,Ababu Crispus Lwasa,3278.0,2020-01-15
1217,Abalyogera,Namulondo,NaN,1941-06-07,2,NaN,I21217,HH101217,Abalyogera Namulondo,NaN,NaT
594,Abalyogera,Namulondo,Omutima,1941-07-13,2,N_ID_6746,NaN,NaN,Abalyogera Namulondo Omutima,2664.0,2019-01-18
655,Abasa,Bukosi,Jajja,1974-05-07,1,NaN,I20655,HH100655,Abasa Bukosi Jajja,NaN,NaT
989,Abasa,Bukosi,Awiti,1974-12-07,1,N_ID_7928,NaN,NaN,Abasa Bukosi Awiti,3058.0,2019-07-14
680,Abatuka,Nahiya,Fanda,1985-02-25,2,N_ID_7004,NaN,NaN,Abatuka Nahiya Fanda,2750.0,2018-08-29
3246,Abatuka,Nahiya,Fanda,1985-02-25,2,NaN,I23246,HH103246,Abatuka Nahiya Fanda,NaN,NaT
3282,Abbo,Shatra,NaN,1934-06-05,2,NaN,I23282,HH103282,Abbo Shatra,NaN,NaT
1393,Abbo,Shatra,Kisanja,1934-04-16,2,N_ID_9137,NaN,NaN,Abbo Shatra Kisanja,3462.0,2022-02-24


In [ ]:
hdss_facility.shape

(7062, 11)

In [ ]:
hdss_facility.duplicated().sum()

2311

In [ ]:
hdss_facility.drop_duplicates(inplace=True)

In [ ]:
hdss_facility.shape

(4751, 11)